In [18]:
import re
import numpy as np
from random import randint
from playwright.async_api import async_playwright, TimeoutError as PlaywrightTimeoutError

In [19]:
SONG_URLS = [f'https://maistocadas.mus.br/{year}' for year in range(2010, 2011)]
SONGS_DIR = 'songs/'
LYRICS_URL = 'https://www.letras.mus.br/'
ACCENTS_MAPPING = {'é': 'e', 'á': 'a', 'í': 'i', 'ó': 'o', 'ú': 'u', 'ã': 'a', 'ç': 'c', 'à': 'a', 'ê': 'e', 'ô': 'o', 'õ': 'o'}

def get_random_geolocation():
    return {'latitude': randint(-90,90), 'longitude':randint(-180,180)}

def get_random_viewport():
    return {'width': randint(800, 1920), 'height': randint(600, 1080)}

def remove_accents(string):
    for accent, replacement in ACCENTS_MAPPING.items():
        string = string.replace(accent, replacement)
    return string

def remove_non_alphanumeric_characters(string):
    return re.sub(pattern=r'[^a-zA-Z0-9\s]', repl='', string=string)

In [30]:
songs_without_lyrics = []
regex = re.compile(r'[^a-zA-Z0-9\s]')

async with async_playwright() as playwright:
    browser = browser = await playwright.chromium.launch(headless = False)
    for url in SONG_URLS:
        song_page = await browser.new_page(viewport=get_random_viewport(), geolocation=get_random_geolocation())
        await song_page.goto(url)
        track_list = await song_page.locator('#tracklist').locator('li').all()
        for track_card in track_list:
            lyrics_page = await browser.new_page(viewport=get_random_viewport(), geolocation=get_random_geolocation())
            await lyrics_page.goto(LYRICS_URL)
            lyrics_page.set_default_timeout(5000)
            song_title = str(await track_card.locator('span.musicas').text_content()).strip()
            song_title_transformed = remove_non_alphanumeric_characters(song_title.replace('&', 'e'))
            artist = str(await track_card.locator('span.artista').text_content()).strip()
            artist_transformed = remove_non_alphanumeric_characters(artist.replace('&', 'e'))
            await lyrics_page.get_by_role('button', name='Consent', exact=True).click()
            await lyrics_page.locator('#main_suggest').fill(song_title + ' ' + artist)
            await lyrics_page.locator("button.header-search-submit").click()
            print(lyrics_page.locator('a.gs-title').nth(0))
            # try:
            #     song_language = await lyrics_page.get_by_role("row", name=f'{artist} - {song_title}').get_by_role("cell").nth(0).text_content()
            # except PlaywrightTimeoutError:
            #     songs_with_no_lyrics.append((song_title, artist))
            # if song_language != 'Portuguese':
            #     await lyrics_page.close()
            #     continue
            # try:
            #     await lyrics_page.get_by_role("row", name=f'{artist} - {song_title}').nth(0).click()
            # except PlaywrightTimeoutError:
            #     print(song_title, artist)


        # title_and_artist_list = [f'"{str(await track_card.locator('span.musicas').text_content()).strip()}","{str(await track_card.locator('span.artista').text_content()).strip()}"\n'
        #                           for track_card in track_list]
        # await song_page.close()
        # with open(SONGS_DIR+url.split('/')[-1]+'.csv', 'w') as f:
        #     f.write('title,artist\n')
        #     f.writelines(title_and_artist_list)

<Locator frame=<Frame name= url='https://www.letras.mus.br/?q=I%20Want%20Know%20What%20Love%20Is%20Mariah%20Carey'> selector='a.gs-title >> nth=0'>
<Locator frame=<Frame name= url='https://www.letras.mus.br/?q=Stereo%20Love%20Edward%20Maya%20%26%20Vika%20Jigulina'> selector='a.gs-title >> nth=0'>


TargetClosedError: Page.goto: Target page, context or browser has been closed
Call log:
navigating to "https://www.letras.mus.br/", waiting until "load"


In [14]:
songs_without_lyrics

[('Tapa na Cara', 'Zezé Di Camargo & Luciano'),
 ('Madri', 'Fernando & Sorocaba'),
 ('Voa Beija Flor', 'Jorge & Mateus'),
 ('Pode Ir Embora', 'Bruno e Marrone'),
 ('Como eu queria te amar', 'Edson'),
 ('Não Mais', 'Victor & Leo'),
 ('Noites De Um Verão Qualquer', 'Skank'),
 ('Rebolation', 'Parangolé'),
 ('Tá Se Achando', 'Guilherme & Santiago'),
 ('Sinônimo de amor', 'Bruno & Marrone'),
 ('Sou Fã', 'Christian & Cristiano'),
 ('Passos escuros', 'HEVO84'),
 ('Sem Me Controlar', 'Marcos & Belutti')]

In [50]:
async with async_playwright() as playwright:
    browser = browser = await playwright.chromium.launch(headless = False)
    lyrics_page = await browser.new_page(viewport=get_random_viewport(), geolocation=get_random_geolocation())
    await lyrics_page.goto(LYRICS_URL)
    await lyrics_page.locator('#google-cse-searchbox-input').fill('Need You Now Lady Antebellum')
    await lyrics_page.locator(":nth-match(button:text('Search'), 1)").click()
    song_language = await lyrics_page.get_by_role("row", name=f'{artist} - {song_title}').get_by_role("cell").nth(0).text_content()
    print(song_language)

TargetClosedError: Target page, context or browser has been closed

In [47]:
songs_with_no_lyrics

[('I Want Know What Love Is', 'Mariah Carey'),
 ('Stereo Love', 'Edward Maya & Vika Jigulina'),
 ('Empire State Of Mind', 'Jay Z ft Alicia Keys'),
 ('TiK ToK', 'Ke$ha'),
 ('Baby', 'Justin Bieber ft. Ludacris'),
 ('Need You Now', 'Lady Antebellum'),
 ('Tapa na Cara', 'Zezé Di Camargo & Luciano'),
 ('Tá vendo aquela Lua', 'Exaltasamba'),
 ('Madri', 'Fernando & Sorocaba'),
 ('Imma Be', 'The Black Eyed Peas'),
 ('Estrela cadente', 'Victor e Leo'),
 ('Voa Beija Flor', 'Jorge & Mateus'),
 ('Love The Way You Lie', 'Eminem ft. Rihanna'),
 ('Só os loucos sabem', 'Charlie brown Jr'),
 ('Hard', 'Rihanna  ft. Jeezy'),
 ('Pode Ir Embora', 'Bruno e Marrone'),
 ('Telephone', 'Lady Gaga ft. Beyoncé'),
 ('Como eu queria te amar', 'Edson'),
 ('Espero A Minha Vez', 'NXZero'),
 ('Doesn’t Mean Anything', 'Alicia Keys'),
 ('Noites De Um Verão Qualquer', 'Skank'),
 ('Ao Vivo e em Cores', 'Victor e Léo'),
 ('Sexy Chick', 'David Guetta Feat. Akon'),
 ('Rebolation', 'Parangolé'),
 ('We No Speak Americano', 'Yol